## Install & Import Requirements

In [ ]:
# این قسمت لطفا فایل مربوط به نقاطی که میخواین تست کنید رو بزارید.
# تو پوشه content
# این فایل باید این ستون ها رو داشته باشد:
#COLS = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'MAX EXTREMUM', 'MIN EXTREMUM', 'Degree']
# برای پیدا کردن درصد دقت ستون
# Degree
# مقدار داشته باشه حتما

test_file = '/content/USA500.IDX_M15_2023.04.17 11.52.53 (OK).xlsx'

In [ ]:
import os
from random import seed as random_seed, shuffle, randint
import copy
from time import sleep
import hashlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.layers import *
import tensorflow as tf

from plotly import graph_objects

from tqdm import tqdm

## Dataset-related Utils

In [ ]:
W = 30  # Size of window
F = 0  # Number of future candles

In [ ]:
DROP_COLS = ['CHART SYMBOL', 'Time Frame', 'i', 'zonesT', 'zonesD', 'Time']
COLS = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'MAX EXTREMUM', 'MIN EXTREMUM', 'Degree']

In [ ]:
def load_df(f_name):
    df = pd.read_excel(f_name)
    df.drop(DROP_COLS, axis=1, inplace=True)

    if list(df.columns) != COLS:
        print("Error: Invalid Excel format.")
        return None

    return df

In [ ]:
def print_stats(f_name):
    df = load_df(f_name)
    data = df.values

    print(f'File    : {f_name}')
    print(f'Min Ext : {np.sum(data[:, 4] != 0)}')
    print(f'Max Ext : {np.sum(data[:, 5] != 0)}')
    print(f'Degree1 : {np.sum(data[:, 6] == 1)}')
    print(f'Degree2 : {np.sum(data[:, 6] == 2)}')
    print()

In [ ]:
def get_valid_datasets():
    all_data_files = list(filter(lambda f: f.endswith('.xlsx'), os.listdir('.')))
    all_data_files = list(filter(lambda f: '50%' not in f, all_data_files))
    return all_data_files

## Sample-related Utils & Config

In [ ]:
def normalize(x, method='z-score'):
    x = x.astype('float32')
    if method == 'z-score':
        mean = x.mean()
        std = x.std()
        x = (x - mean) / std
        return x
    assert False, "Invalid Normalization Method."

In [ ]:
def extract_window(data_array, target_index):
    i = target_index
    data = data_array[i - (W - F - 1) : i + (F + 1), :4]
    return data

In [ ]:
def extract_features(window_data, is_max, only_close, reverse_max, diff):
    if is_max: assert reverse_max
    assert window_data.shape == (W, 4)
    if only_close:
        window_data = window_data[:, 3:]
    if is_max:
        window_data = - window_data
    if diff:
        diff = np.zeros_like(window_data)
        diff[1:, :] =  window_data[1:, :] - window_data[:-1, :]
        window_data = diff
    return normalize(window_data)

In [ ]:
def find_target_indices(file_path=None):
    target_indices_all = []
    file_path_list = get_valid_datasets() if file_path is None else [file_path]
    for f_name in file_path_list:
        data = load_df(f_name).values
        len_sequence = data.shape[0]
        degree = data[:, 6]
        target_indices = np.argwhere((degree == 1.0) | (degree == 2.0))
        target_indices = target_indices.flatten().tolist()
        target_indices = list(filter(lambda i: W-F-1 <= i <= len_sequence-1-F, target_indices))
        target_indices = list(filter(lambda i: data[i, 4] != data[i, 5], target_indices))
        target_indices = list(map(lambda i: {
            'dataset': f_name,
            'index': i,
            'label': data[i, 6] == 1.0,
            'is_max': data[i, 4] != 0
        }, target_indices))
        target_indices_all.extend(target_indices)
    return target_indices_all

In [ ]:
def split_list(data, first_percent=80, seed=10):
    data = copy.deepcopy(data)
    random_seed(seed)
    shuffle(data)
    i = int(len(data) * (first_percent / 100))
    list1 = data[:i]
    list2 = data[i:]
    return list1, list2

In [ ]:
def generate_data(datasets, indices_list, only_close, reverse_max, diff, augment_flip_x, seed=10, no_shuffle=False):
    res_x = []
    res_y = []
    for indice in indices_list:
        if indice['is_max'] and (not reverse_max): continue
        window_data = extract_window(datasets[indice['dataset']], indice['index'])
        features = extract_features(window_data, indice['is_max'], only_close, reverse_max, diff)
        res_x.append(features)
        res_y.append(indice['label'])
        if augment_flip_x:
            features = np.flip(features, axis=0)
            if diff:
                features[:-1, :] = features[1:, :]
            res_x.append(features)
            res_y.append(indice['label'])
    merged = list(zip(res_x, res_y))
    if no_shuffle:
        random_seed(seed)
        shuffle(merged)
    res_x, res_y = list(zip(*merged))
    res_x = np.array(res_x, dtype='float32')
    res_y = np.array(res_y)
    return res_x, res_y

## Load & Inference

In [ ]:
!rm -rf best_model.h5
!gdown 1Aq07QeNDC0tCHh7V-dru0K8-ptD6w_Ce

Downloading...
From: https://drive.google.com/uc?id=1Aq07QeNDC0tCHh7V-dru0K8-ptD6w_Ce
To: /content/best_model.h5
100% 130k/130k [00:00<00:00, 127MB/s]


In [ ]:
model = keras.models.load_model('best_model.h5')
datasets = {name: load_df(name).values for name in [test_file]}
test_indices = find_target_indices(test_file)
x_test, y_test = generate_data(datasets, test_indices, False, True, False, False)
y_pred = model.predict(x_test, verbose=False)

accuracy = 0
for i in range(len(test_indices)):
    # Check data validity
    assert y_test[i] == test_indices[i]['label']

    index = test_indices[i]["index"]
    is_max = 'Y' if test_indices[i]["is_max"] else 'N'
    grade = int(test_indices[i]["label"])
    grade_pred = int(y_pred[i][0] >= 0.5)
    print(f'Index: {index}  -  Max: {is_max}  -  Grade: {grade}  -  Prediction: {grade_pred}')
    accuracy += int(grade == grade_pred)
accuracy = (accuracy / len(test_indices)) * 100
print(f'Accuracy: {accuracy:.2f}%')

Index: 37  -  Max: Y  -  Grade: 1  -  Prediction: 0
Index: 41  -  Max: N  -  Grade: 1  -  Prediction: 1
Index: 43  -  Max: Y  -  Grade: 1  -  Prediction: 1
Index: 48  -  Max: N  -  Grade: 1  -  Prediction: 1
Index: 49  -  Max: Y  -  Grade: 1  -  Prediction: 0
Index: 51  -  Max: N  -  Grade: 1  -  Prediction: 1
Index: 63  -  Max: Y  -  Grade: 1  -  Prediction: 1
Index: 67  -  Max: N  -  Grade: 1  -  Prediction: 0
Index: 69  -  Max: Y  -  Grade: 1  -  Prediction: 0
Index: 77  -  Max: N  -  Grade: 1  -  Prediction: 1
Index: 80  -  Max: Y  -  Grade: 0  -  Prediction: 0
Index: 82  -  Max: N  -  Grade: 0  -  Prediction: 1
Index: 88  -  Max: Y  -  Grade: 1  -  Prediction: 0
Index: 90  -  Max: N  -  Grade: 1  -  Prediction: 1
Index: 104  -  Max: Y  -  Grade: 0  -  Prediction: 0
Index: 111  -  Max: N  -  Grade: 0  -  Prediction: 1
Index: 116  -  Max: Y  -  Grade: 1  -  Prediction: 1
Index: 120  -  Max: N  -  Grade: 0  -  Prediction: 1
Index: 122  -  Max: Y  -  Grade: 0  -  Prediction: 0
Index: 